Setup 

In [ ]:
from __future__ import print_function
from __future__ import division

from glob import glob
import os.path as op 
import argparse , time , tarfile 
import multiprocessing as mpl 
from hashlib import md5 

import os   
import tarfile 
import re 
import newspaper

In [ ]:
# Tạo một đối tượng ArgumentParser để chứa các đặc tả đối sốvaf các tùy chọn áp dụng cho bộ 
# phân tích 
parser = argparse.ArgumentParser()
# Thêm 1 đối số tùy chọn có tên là --html_archive, kiểu là chuỗi và giá trị mặc định là 
# "openwebtext/RS_2017-04-4_data.xz". Đối tượng này sẽ được lưu trong thuộc tính html_archive 
parser.add_argument("--html_archive", type=str, default="openwebtext/RS_2017-04-4_data.xz")
# Thêm một đôi số tùy chọn có tên là --chunk_size kiểu số nguyên và giá trị mặc định 
# là 100 . Đối số anyf được lưu trong thuộc tính --chunk_size của đối tượng kết quả
parser.add_argument("--chunk_size", type=int, default=100)
# Thêm một đối số tùy chọn có tên là --n_procs kiểu số nguyên và giá trị mặc định là 5 
# Đối số này được lưu trong thuộc tính n_procs của đối tượng kết quả 
parser.add_argument("--n_procs", type=int, default=5)
# Thêm một đối số tùy chọn có tên là --output_dir, kiểu str và giá trị mặc định là parsed. 
# Đối số này được lưu trong thuộc tính output_dir của đối tượng kêt quả
parser.add_argument("--output_dir", type=str , default="parsed")
# Chạy bộ phân tích và đặt dữ liệu được trích xuất trong một đối tượng argpase.Namespace. Đối 
# tượng này có các thuộc tính tương ứng với các đối số được định nghĩa 
args = parser.parse_args()

In [ ]:
# Thiết lập phương thức parse_file . Nhận một đối số là một tuple gồm tên file và nội dung HTML 
# của file đó . Hàm này sẽ tạo một hash có nội dung HTML bằng thuật toán MD5 , và sử dụng hash đó 
# làm url cho một đối tuowngj Artical của thư viện newspaper. 
def parse_file(file_entry):
    # Gán filename và html bằng 2 phần tử của tuple file entry. 
    file_name , html = file_entry
    # Tạo một hash của nội dung HTML bằng thuật toán MD5 và lưu vào biến url_hash 
    url_hash = md5(html).hexdigest()
    # Tạo một đối tượng Article của thư viện newspaper với url là url_hash và không tải ảnh 
    article = newspaper.Article(url = url_hash, fetch_image=False)
    # Thiết lập nội dung HTML cho đối tượng Article bằng phương thức set_html 
    article.set_html(html)
    # Phân tích văn bản HTML bằng phương thức parse 
    article.parse()
    # Trả về 1 tuple gồm file và văn bản đã được phân tích 
    return (file_name, article.text)


# Thiết lập phương thức save_parsed_text hàm này nhận hai đối ố là một dnah scahs các typle bao gồm 
# tên các file và văn bản đã được phân tích , và một đường dẫn thư mục đầu ra. 
# Hàm này sẽ duyệt qua danh sách các tuplevaf lưu văn bản vào file có cùng tên với file HTML ban đầu 
# trong thư mục đầu ra .
def save_parsed_text(parsed_entries, out_dir):
    # duyệt qua 1 danh sách tuple parsed_entries lấy ra file name và văn bản đã được phân tich 
    for fn , txt in parsed_entries: 
        # Sử dụng hàm join để nối đường dẫn đầu ra out_dir với tên file fn 
        txt_fp = op.join(out_dir, fn)
        # mở file văn bản ở chế độ ghi, gán đối tượng cho biến handle
        with open(txt_fp, "w") as handle:
            # ghi nội dung của biến txt vào file  vào file văn bản qua biến handle 
            handle.write(txt)


# Thiết lập phương thức get_processedfiles  Hàm này nhận một đối số alf một đường dẫn thư mục đầu ra. Hàm này sẽ trả về 
# một tập hợp các tên file đã được trích xuất văn bản có đuôi txt . Hàm này sử dụng glob để lấy 
# tất cả các file có định dạng txt 
def get_processed_files(out_dir):
    # nối các file đầu ra đã được xử lý với định dnagj đuôi txt 
    # sau đó sử dụng hàm glob để tìm tất cả các tệp có định dạng .txt 
    parsed = glob(op.join(out_dir, "*.txt")) 
    # Trả về 1 tập hợp các tên file đã được trích xuất văn bản bằng cáchlấy phần cuối cùng
    # của đường dẫn file sau khi tách bằng hàm op.split 
    return set([op.split(f)[-1] for f in parsed])


In [ ]:
# Thiết lập phương thức parse_archive có chức năng được sử dụng để trích xuất các 
# file HTML trong một file nén và lưu văn bản vào các file trong thư mục đầu ra 
# Hàm này nhận 4 đối số là đường dẫn file nén chứa các file HTML archive_fb , đường
# dẫn đến thư mục đầu ra out_dir, số lượng tiến trình song song n_procs, và kích thước mỗi đoạn 
# file chunk_size 
def parse_archive(archive_fb , out_dir, n_procs, chunk_size=100):
    # gọi hàm get_parsed_files để lấy tập hợp tên các file đã được trích xuất văn bản 
    # từ đường dẫn out_dir
    processed = get_processed_files(out_dir=out_dir)
    # mở file và gán nó cho biến tf các file HTML có định dạng tar nên ta dùng tarfile.OPEN
    with tarfile.Open(archive_fb, "r") as tf: 
        # lấy ra dnah sách tên file trong file nén bằng hàm tf.getnames 
        # bỏ qua các tên file đax được tríc xuất văn bản 
        files = list(set(tf.getnames()) - set(processed))
        # kiểm tra xem só lượng file trong dnah sách files == 0:
        if len(files) == 0:
            # ta quay trở lại 
            return 
        # Và kiểm tra xem số lượng file đã được tríc xuất vaen bản > 0
        if len(processed) > 0:
            # in ra thông báo cho biết số file đã được sử lý trước
            print("{} files already processed.". format(len(processed)))

        # duyệt qua số lượng files từ danh sách file html đã được trích xuất
        # chia nó thành các phần với kích thước chunk_size 
        for ci , chunk in enumerate(chunks(files, chunk_size)):
            # Tạo một danh sách các tuple gồm tên file và nội dung HTML của nó 
            # bằng cách sử dụng hàm tf.extractfile để đọc nội dung từ file nén,
            file_entries = [(fn , tf.extracfile(fn).read()) for fn in chunk]

            # lấy thời gian hiện tại lưu vào biến t1 
            t1 = time.time()
            # Tạo một danh sách các tuple gồm tên file và văn bản đã được phân tích bằng cách 
            # sử dụng hàm map và hàm parse_file để áp dụng parse_file cho mỗi phần tử 
            # trong danh scahs file_entries 
            parsed = list(map(parse_file, file_entries))

            # loại bỏ đi typle có văn bản rỗng  
            parsed = [p for p in parsed if len(p)[1] != 0]

            # Tính tỷ lệ thành công của việc trích xuất văn bản , bằng cách chia số lượng 
            # phần tử của dnah sách parsed cho số lượng phần tử của danh sách chunk * 100 
            hit_rate = len(parsed) / len(chunk) * 100
            #In ra thời gian đã mất để phân tích đoạn file hiện tại, và tỷ lệ thành công của việc trích xuất văn bản
            print("Parsing chunk {} took {} seconds".format(ci + 1, time.time() - t1))
            print(" -- {}% of chunk {}'s docs yielded text.".format(hit_rate, ci + 1))

            # lấy thời gian hiện tại lưu vào biến t1
            t1 = time.time()
            # Gọi hàm save_parsed_text để lưu các tupel trong danh sách parsed vào các file văn bản 
            # trong thư mục đầu ra 
            save_parsed_text(parsed, out_dir)
            # In ra thời gian đã mất để lưu đoạn file hiện tại.
            print("Saving chunk {} took {} seconds".format(ci + 1, time.time() - t1))



def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i : i + n]   

In [ ]:
# Thiết lập phương thức extract_month . Hàm này là một hàm để trích xuất thánh từ tên file chứa url 
# Hàm này nhận một đối số là tên file và trả về chuỗi là tháng 
def extract_month(url_file_name):
    # Tạo một biểu thức chính quy month_re để khớp với các chuỗi có dạng RS_ theo sau là 4 chữ số,
    # dấu gạch ngang, và 2 chữ số. Ví dụ: RS_2017-04.
    month_re = r"(RS_.*2\d{3}-\d{2})"
    # tách tên file từ đường dẫn url và lưu nó vào biến month 
    month = op.split(url_file_name)[-1]
    # Áp dụng biểu thức chính quy month_re cho biến month  bằng hàm re.match 
    # và lấy kết quả khớp bằng phương thức group lưu lại vào biến month 
    month = re.match(month_re, month).group()
    # trả về month là kết qủa của hàm 


# Thiết lập phương thức extract_archive Hàm này nhận hai đối số là đường dẫn đến file nén
# chứa các file HTML (archive_fp), và đường dẫn đến thư mục đầu ra (outdir). 
def extract_archive(archive_fb,outdir="."):
    # sử dụng tarfile.Open để mở cac file HTML có định dạng .tar gán cho biến tar
    with tarfile.Open(archive_fb, "r") as tar:
        # giải nén tất acr accs file trong file nén vào thư mục đầu ra bằng 
        # tar.extractall 
        tar.extractall(outdir)
    
    return outdir 

# Thiết lập phương thức makedir để kiểm tra và tạo thư mục 
def mkdir(fp):
    # kiểm tra xem fp có phải là thư mục đã tồn tại 
    if not op.exits(fp):
        # nếu không sử dụng makedir để biến nó thành 1 thư mục 
        os.makedirs(fp)
    return fp 



# kiểm tra xem file Python có được chạy trực tiếp hay không 
# nếu có khối lệnh trong nó sẽ được thực hiện 
if __name__ == "__main__":
    # Gọi hàm extract_month để trích xuất tháng từ tên file chứa url, lưu vào biến month.
    month = extract_month(args.html_archive)
    # : Gọi hàm mkdir để tạo một thư mục đầu ra có tên là tháng đã trích xuất, lưu vào biến out_dir.
    out_dir = mkdir(op.join(args.output_dir, month))
    # Gọi hàm parse_archive để trích xuất văn bản từ các file HTML trong file nén, 
    # và lưu văn bản vào các file trong thư mục đầu ra.
    parse_archive(args.html_archive, out_dir, args.n_procs, args.chunk_size)
    #  In ra màn hình chuỗi “Done!” để báo hiệu quá trình trích xuất văn bản đã hoàn tất.
    print("Done!")